In [29]:
import sys
from pathlib import Path
import numpy as np
from scipy.ndimage import center_of_mass
#import warnings
#warnings.filterwarnings("error")
import matplotlib.pyplot as plt
import pandas as pd
import plotly.express as px
import numpy as np
from superpose3d import Superpose3D


PIPELINE_ROOT = Path('../src').resolve().parent.parent
sys.path.append(PIPELINE_ROOT.as_posix())
print(PIPELINE_ROOT)

from library.registration.brain_structure_manager import BrainStructureManager
from library.registration.algorithm import umeyama
from registration.pointset_alignment import get_rigid_aligned_points, get_similarity_alignment_points

/home/eddyod/programming/preprocessing-pipeline/src


In [2]:
def brain_to_atlas_transform(brain_coord, r, t):
    brain_coord = np.array(brain_coord).reshape(3, 1) # Convert to a column vector
    atlas_coord = r @ brain_coord + t
    return atlas_coord.T[0] # Convert back to a row vector

def calculate_distance(com1, com2):
    return (np.linalg.norm(com1 - com2))
        
def plot_point_sets_3d(point_sets):
    df = pd.DataFrame()
    for data, label in point_sets:
        df_cur = pd.DataFrame(data.T, columns=['x', 'y', 'z'])
        df_cur['label'] = label
        #df = df.concat(df_cur, ignore_index=True)
        df = pd.concat([df, df_cur], axis= 0)
    
    fig = px.scatter_3d(df, x='x', y='y', z='z', color='label', width=800, height=600)
    fig.update_layout(
    scene={
        'xaxis': {'range': (0, 528)}, 
        'yaxis': {'range': (0, 320)},
        'zaxis': {'range': (0, 456)},
    })
    return fig
        

In [3]:
animal = 'Atlas'
brain = BrainStructureManager(animal)
brain.fixed_brain = BrainStructureManager('Allen')

In [4]:
brainstem_keys = set(brain.allen_structures_keys) - brain.midbrain_keys 

In [5]:
moving_coms = brain.get_coms(annotator_id=1)
allen_coms = brain.fixed_brain.get_coms(annotator_id=1)
common_keys = sorted(allen_coms.keys() & moving_coms.keys())
allen_points = np.array([allen_coms[s] for s in common_keys])
moving_points = np.array([moving_coms[s] for s in common_keys])

In [6]:
common_midbrain_keys = sorted(allen_coms.keys() & moving_coms.keys() & brain.midbrain_keys)
allen_midbrain_points = np.array([allen_coms[s] for s in common_midbrain_keys])
moving_midbrain_points = np.array([moving_coms[s] for s in common_midbrain_keys])

In [7]:
allen_point_dict = {s:allen_coms[s] for s in common_keys}
moving_point_dict = {s:moving_coms[s] for s in common_keys}

In [30]:
reg_dict = get_similarity_alignment_points(allen_point_dict, moving_point_dict)

In [24]:
moving_points.shape

(37, 3)

In [ ]:
brainstem_keys

In [ ]:
allen_point_dict['SC']

In [ ]:
moving_point_dict['SC']

In [ ]:
distances = []
sortme = {}
for structure in common_keys:
    (x,y,z) = allen_point_dict[structure]
    allen_point = np.array([x,y,z])    
    moving_point = np.array(moving_point_dict[structure])
    #print(atlas_point, allen_point)
    d = calculate_distance(allen_point, moving_point)
    distances.append(d)
    sortme[structure] = d
print(f'n={len(distances)}, min={min(distances)} max={max(distances)}, mean={np.mean(distances)}')
ds = {k: v for k, v in sorted(sortme.items(), key=lambda item: item[1])}
for structure, d in ds.items():
    print(f'{structure} distance from Allen={round(d,2)} micrometers')

In [ ]:
plot_point_sets_3d([
    (moving_points.T/25, 'unaligned moving centers'),
    (allen_points.T/25, 'Allen centers')
])

In [ ]:
(RMSD, R1, t1, _) = Superpose3D(allen_points, moving_points, allow_rescale=True)
t1 = t1.reshape(3,1)
print(RMSD)

In [ ]:
allen_midbrain_points.shape

In [25]:
R, t = umeyama(moving_points.T, allen_points.T)

In [26]:
t

array([[ -8.78746122],
       [ 43.28687301],
       [-57.35439132]])

In [27]:
reg_points = R @ moving_points.T + t

 That does not preclude starting the new idea but is just to say that somehow we need to get David's attention on what you have done- do a graph
= box plot of the midbrain com distances vs allen (active atlas based)
=box plot of the brainstem vs allen distances (but not using midbrain values to improve the distances)
This should provide an easy visual...


When the tethering is done for the brainstem structures though- he does not want the midbrain metrics to be included ( I don't know though if he means that the alignment is done with midbrain and then the evaluated outputs are only for the brainstem structures...?) your guess is best
So- in any case it was clear that he did not want metrics for midbrain and brainstem structures to be combined.

In [ ]:
print('Mean')
print(np.mean(allen_points, axis=0))
print(np.mean(reg_points, axis=1))
print('Min')
print(np.min(allen_points, axis=0))
print(np.min(reg_points, axis=1))
print('Max')
print(np.max(allen_points, axis=0))
print(np.max(reg_points, axis=1))

In [ ]:
reg_point_dict = {s:moving_coms[s] for s in common_keys}

In [31]:
distances = []
distances2 = []
sortme = {}
for structure in common_keys:
    (x,y,z) = allen_point_dict[structure]
    sitk_point = reg_dict[structure]
    allen_point = np.array([x,y,z])    
    moving_point = np.array(moving_point_dict[structure])
    reg_point = brain_to_atlas_transform(moving_point, R, t)
    d = calculate_distance(allen_point, reg_point)
    ds = calculate_distance(allen_point, sitk_point)
    distances2.append(ds)
    distances.append(d)
    print(f'{structure} COM={reg_point.round()} SITK={round(ds,2)} distance={round(d,2)}')
    sortme[structure] = d

print(f'n={len(distances)}, min={min(distances)} max={max(distances)}, mean={np.mean(distances)}')
print(f'n={len(distances2)}, min={min(distances2)} max={max(distances2)}, mean={np.mean(distances2)}')
#ds = {k: v for k, v in sorted(sortme.items(), key=lambda item: item[1])}
#for structure, d in ds.items():
#    print(f'{structure} {round(d,2)}')

3N_L COM=[9054. 3980. 5623.] SITK=219.52 distance=216.77
3N_R COM=[9073. 4016. 5885.] SITK=228.6 distance=225.06
4N_L COM=[9578. 4030. 5447.] SITK=256.96 distance=252.76
4N_R COM=[9550. 4063. 5980.] SITK=295.14 distance=291.58
5N_L COM=[10161.  5211.  4201.] SITK=141.25 distance=136.76
5N_R COM=[10102.  5158.  7182.] SITK=194.94 distance=191.97
6N_L COM=[10775.  5319.  5259.] SITK=107.47 distance=108.67
6N_R COM=[10781.  5366.  6027.] SITK=165.42 distance=165.75
7N_L COM=[10780.  6479.  4476.] SITK=341.2 distance=334.35
7N_R COM=[11357.  6575.  7059.] SITK=541.68 distance=542.9
7n_L COM=[10538.  5651.  4538.] SITK=277.44 distance=281.11
7n_R COM=[10558.  5587.  6790.] SITK=198.84 distance=202.02
AP COM=[12708.  5525.  5518.] SITK=543.42 distance=545.88
Amb_L COM=[11660.  6408.  4292.] SITK=314.13 distance=307.78
Amb_R COM=[11678.  6238.  7012.] SITK=440.33 distance=433.63
DC_L COM=[11301.  4923.  3256.] SITK=90.89 distance=89.33
DC_R COM=[11072.  4778.  8146.] SITK=301.41 distance=300.

In [ ]:
reg_points.shape

In [ ]:
plot_point_sets_3d([
    (reg_points/25, 'registered moving centers'),
    (allen_points.T/25, 'Allen centers')    
])